In [1]:
# Imports (numpy, pandas, matplotlib, seaborn imported automatically)
from etherscan import Etherscan
from BondingCurveNexus import config

In [25]:
# bring in csvs

# historic active cover amount
active_cover_amount = pd.read_csv('../raw_data/active_cover_amount.csv')
active_cover_amount.columns = ['date', 'USD', 'ETH']
active_cover_amount['date'] = pd.to_datetime(active_cover_amount['date'])

# claims history
claims_history = pd.read_csv('../raw_data/all_claims.csv')
claims_history['timestamp'] = pd.to_datetime(claims_history['timestamp'])

# cover history
cover_history = pd.read_csv('../raw_data/all_covers.csv')

In [10]:
active_cover_amount

,date,USD,ETH
0,2019-07-12 12:39:36,9.775000e+02,3.549383
1,2019-07-13 09:38:17,2.807950e+04,103.606745
2,2019-07-14 14:56:50,4.852530e+04,204.136553
3,2019-07-14 15:03:03,7.119130e+04,304.197325
4,2019-07-15 09:08:31,7.920615e+04,349.341287
...,...,...,...
13569,2022-05-16 10:35:53,3.647409e+08,179479.028714
13570,2022-05-16 10:36:04,3.648128e+08,179127.486414
13571,2022-05-16 11:51:59,3.643953e+08,179264.858265
13572,2022-05-16 12:02:58,3.638176e+08,179537.982378


In [14]:
accepted_claims = claims_history[claims_history['verdict'] == 'Accepted']

In [23]:
days_since_first_claim = (pd.Timestamp.now() - accepted_claims['timestamp'].iloc[0]).days

In [26]:
avg_claim_freq = accepted_claims['verdict'].count() / days_since_first_claim

In [27]:
avg_claim_freq 

0.03178484107579462

In [28]:
cover_history

,block_number,start_time,premium,address,cover_id,end_time,currency,amount,project,amount_usd,premium_usd
0,8667271,2019-10-03 04:56:05,0.012991,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,84,2020-10-02 04:56:05,ETH,1.0,Uniswap v1,2012.69,26.147061
1,8759426,2019-10-17 15:55:29,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,93,2020-03-15 15:55:29,ETH,12.0,Uniswap v1,24152.28,128.944411
2,8873386,2019-11-04 20:17:23,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,98,2020-04-02 20:17:23,ETH,12.0,Uniswap v1,24152.28,128.944411
3,8895454,2019-11-08 09:33:17,0.074743,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,100,2020-04-06 09:33:17,ETH,14.0,Uniswap v1,28177.66,150.435146
4,8935109,2019-11-14 23:06:33,0.172977,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,106,2020-02-12 23:06:33,ETH,54.0,Uniswap v1,108685.26,348.149909
...,...,...,...,...,...,...,...,...,...,...,...
7329,12246810,2021-04-15 20:17:13,448.459959,0xcb876f60399897db24058b2d58d0b9f713175eef,3901,2021-07-14 20:17:13,DAI,70000.0,Pool Together v3,69965.00,448.235729
7330,11785120,2021-02-03 19:16:33,1.127023,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3150,2022-02-03 19:16:33,ETH,7.0,Set Protocol v2,14088.83,2268.348109
7331,11812010,2021-02-07 22:24:11,4.015933,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3198,2022-02-07 22:24:11,ETH,25.0,Set Protocol v2,50317.25,8082.827736
7332,11965147,2021-03-03 12:02:21,0.134255,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3531,2021-06-01 12:02:21,ETH,10.0,Set Protocol v2,20126.90,270.213344
